<a href="https://colab.research.google.com/github/Israel-Ferreira/extract-boston-311-files/blob/main/save_data_blob_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Criando uma pasta e extraindo os dados

In [1]:
%mkdir -p data

In [2]:
import urllib.request

links_arquivos =  [
    ("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv", "data/dados_2015.csv"),
    ("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv", "data/dados_2016.csv"),
    ("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv", "data/dados_2017.csv"),
    ("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv", "data/dados_2018.csv"),
    ("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv", "data/dados_2019.csv"),
    ( "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv", "data/dados_2020.csv")
]


In [3]:
import requests


def extract_data(file_url, filename):
  headers = {"User-Agent": "Mozilla/5.0"}
  response =requests.get(file_url, headers=headers)

  response.raise_for_status()

  with open(filename, "wb") as f:
    f.write(response.content)


In [ ]:
for file_tuple in links_arquivos:
  file_url, filename =  file_tuple
  print(file_url)
  extract_data(file_url, filename)

## Compactando os arquivos

In [5]:
import shutil

def zipfile(path, filename):
  shutil.make_archive(filename, 'zip', path)

In [7]:
zipfile("/content/data", "dados_boston")

## Conectando a conta de armazenamento

In [ ]:
%pip install azure-storage-blob azure-identity

In [10]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

import os
from google.colab import userdata

In [11]:
def get_blob_service_client():
  account_url = "https://dadosboston1204.blob.core.windows.net"
  shared_access_key = userdata.get('SHARED_ACCESS_KEY')
  return BlobServiceClient(account_url, credential=shared_access_key)




In [13]:
blob_client = get_blob_service_client()

## Fazendo upload do arquivo compactado no Blob Storage

In [14]:
def upload_blob_file(blob_service_client, container_name, filepath, filename):
  container_client = blob_service_client.get_container_client(container=container_name)

  with open(file=filepath, mode="rb") as data:
    blob_client = container_client.upload_blob(name=filename, data=data, overwrite=True)

In [15]:
upload_blob_file(blob_client, "servicosbostonzip", "/content/dados_boston.zip", "dados_boston.zip")